# Análise de Imagens de Dígitos de 0 a 9

Nosso problema consiste em analisar imagens de dígitos escritos à mão, cada uma com uma resolução de 28x28 pixels.

Cada um dos 784 pixels pode variar de 0 a 255, onde 0 representa a cor preta e 255 representa a cor branca.

## Representação de Dados

Para representar esses dígitos em nossa rede neural, organizaremos os dados em uma matriz. Essa matriz terá 784 linhas, onde cada linha representa uma característica de pixel. No entanto, para a entrada em nossa rede neural, faremos a transposição dessa matriz, transformando as linhas em colunas, de modo que cada coluna represente um pixel.

## Camadas da Rede Neural

Nossa rede neural será composta por três camadas:

1. **Camada de Entrada (Input Layer):** Esta camada receberá as 784 características como entrada.

2. **Camada Oculta (Hidden Layer):** A segunda camada, chamada de camada oculta, processará os dados intermediários.

3. **Camada de Saída (Output Layer):** A terceira e última camada, chamada de camada de saída, fornecerá o resultado final.

## Processo nas Camadas

### Camada de Entrada:

- $A^{[0]} = \underbrace{x}_\text{784 \; x \; m}$.

### Camada Oculta:

- $\underbrace{Z^{[1]}}_\text{10 x m} = \underbrace{w^{[1]}}_\text{10 x 784} \cdot \underbrace{A^{[0]}}_\text{784 x m} + \underbrace{b^{[1]}}_\text{10 x 1}$ onde Z fará alterações utilizando W (peso) e b (bias).
- Função de Ativação: ReLU (Rectified Linear Unit)

  $$
  A^{[1]} = \begin{cases}
    x, & \text{se $x>0$}.\\
    0, & \text{se $x\leq 0$}.
  \end{cases}
  $$

### Camada de Saída:

- $\underbrace{Z^{[2]}}_\text{10 x m} = \underbrace{w^{[2]}}_\text{10 x 784} \cdot \underbrace{A^{[1]}}_\text{784 x m} + \underbrace{b^{[2]}}_\text{10 x 1}$ onde Z fará alterações utilizando W (peso) e b (bias).

- Função de Ativação: Softmax

  $$
  A^{[2]} = \text{softmax}(Z^{[2]})
  $$

## Treinamento da Rede Neural

Após passar pelas camadas, calcularemos os erros dos parâmetros para, em seguida, atualizar esses parâmetros. Esse processo será repetido até que as previsões da rede estejam satisfatórias.



<img title="a title" alt="Alt text" src="data/rede neural.png">

In [28]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [29]:
data = pd.read_csv('data/train.csv')

In [30]:
data

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41996,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41997,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41998,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [31]:
# Converte os dados em um array NumPy para facilitar o manuseio.
data = np.array(data)

# Obtém o número de exemplos de treinamento (m) e o número de características (n).
m, n = data.shape

# Embaralha os dados para garantir aleatoriedade no conjunto de treinamento.
np.random.shuffle(data)

# Divide os dados em conjunto de teste (teste) e treinamento (train).
# Conjunto de teste (teste) contendo 1000 exemplos.
data_teste = data[0:1000].T
Y_teste = data_teste[0]
X_teste = data_teste[1:n]
X_teste = X_teste / 255.

# Conjunto de treinamento (train) contendo os exemplos restantes.
data_train = data[1000:m].T
Y_train = data_train[0]
X_train = data_train[1:n]
X_train = X_train / 255.

# Obtém o número de exemplos no conjunto de treinamento.
_, m_train = X_train.shape


In [32]:
def init_params():
    # Inicialização dos parâmetros W (pesos) e b (bias) de forma aleatória entre -0.5 e 0.5.
    W1 = np.random.rand(10, 784) - 0.5
    b1 = np.random.rand(10, 1) - 0.5
    W2 = np.random.rand(10, 10) - 0.5
    b2 = np.random.rand(10, 1) - 0.5
    return W1, b1, W2, b2

def ReLU(Z):
    # Função de ativação ReLU (Rectified Linear Unit).
    return np.maximum(Z, 0)

def softmax(Z):
    # Função de ativação Softmax para calcular as probabilidades das classes.
    A = np.exp(Z) / sum(np.exp(Z))
    return A

def forward_prop(W1, b1, W2, b2, X):
    # Propagação direta (forward propagation) na rede neural.
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2

def ReLU_deriv(Z):
    # Derivada da função ReLU.
    return Z > 0

def one_hot(Y):
    # Conversão de rótulos de classe em codificação one-hot.
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y):
    # Propagação inversa (backward propagation) para calcular gradientes.
    one_hot_Y = one_hot(Y)
    dZ2 = A2 - one_hot_Y
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2)
    dZ1 = W2.T.dot(dZ2) * ReLU_deriv(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1)
    return dW1, db1, dW2, db2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    # Atualização dos parâmetros W (pesos) e b (bias) utilizando gradiente descendente.
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1    
    W2 = W2 - alpha * dW2  
    b2 = b2 - alpha * db2    
    return W1, b1, W2, b2


In [33]:
def get_predictions(A2):
    # Obtém as previsões finais selecionando a classe com a probabilidade mais alta (argmax).
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    # Calcula a precisão comparando as previsões com os rótulos reais.
    print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X, Y, alpha, iterations):
    # Função para realizar o processo de treinamento da rede neural usando gradiente descendente.
    W1, b1, W2, b2 = init_params()  # Inicializa os parâmetros da rede.
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)  # Propagação direta (forward propagation).
        dW1, db1, dW2, db2 = backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y)  # Propagação inversa (backward propagation).
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)  # Atualiza os parâmetros.
        if i % 10 == 0:
            print("Iteration: ", i)
            predictions = get_predictions(A2)  # Obtém as previsões.
            print(get_accuracy(predictions, Y))  # Calcula e exibe a precisão.
    return W1, b1, W2, b2


In [34]:
def test(W1, b1, W2, b2):
    # Função para testar o desempenho da rede neural nos dados de teste.
    # Propagação direta (forward propagation) nos dados de teste.
    _, _, _, A2 = forward_prop(W1, b1, W2, b2, X_teste)
    # Obtém previsões com base nos resultados da propagação direta.
    predictions = get_predictions(A2)
    # Calcula e imprime a precisão das previsões em relação aos rótulos reais.
    print("Accuracy on Test Data:", get_accuracy(predictions, Y_teste))

def only_predictions(W1, b1, W2, b2, X):
    # Função para obter apenas as previsões da rede neural para um conjunto de entrada.
    # Propagação direta (forward propagation) nos dados de entrada.
    _, _, _, A2 = forward_prop(W1, b1, W2, b2, X)
    # Imprime as probabilidades de classe para cada exemplo.
    print("Probabilities of Class Predictions:")
    print(A2)
    # Obtém e imprime as previsões finais com base nas probabilidades.
    predictions = get_predictions(A2)
    print()
    print("Class Predictions:")
    print(predictions)


In [35]:
pred = data.T  # Transpõe a matriz de dados para que as características sejam colunas e exemplos sejam linhas.
pred_X = pred[1:n] / 255  # Normaliza as características, dividindo por 255 para escala entre 0 e 1.
pred_Y = pred[0] / 255  # Normaliza os rótulos, dividindo por 255 para escala entre 0 e 1.

# Seleciona um único exemplo de teste (primeira coluna dos dados normalizados).
X = pred_X[:, 0].reshape(784, 1)


In [36]:
W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 0.6, 2000)

Iteration:  0
[5 5 7 ... 6 7 4] [6 9 8 ... 1 4 8]
0.06314634146341463
Iteration:  10
[9 9 9 ... 1 7 1] [6 9 8 ... 1 4 8]
0.464
Iteration:  20
[6 9 8 ... 1 7 1] [6 9 8 ... 1 4 8]
0.6356341463414634
Iteration:  30
[6 7 8 ... 1 7 1] [6 9 8 ... 1 4 8]
0.6481707317073171
Iteration:  40
[6 9 8 ... 1 7 2] [6 9 8 ... 1 4 8]
0.7375853658536585
Iteration:  50
[6 9 8 ... 1 9 1] [6 9 8 ... 1 4 8]
0.7732682926829268
Iteration:  60
[6 9 8 ... 1 9 2] [6 9 8 ... 1 4 8]
0.7960243902439025
Iteration:  70
[6 9 8 ... 1 9 2] [6 9 8 ... 1 4 8]
0.8138048780487804
Iteration:  80
[6 9 8 ... 1 4 2] [6 9 8 ... 1 4 8]
0.8285121951219512
Iteration:  90
[6 9 8 ... 1 4 2] [6 9 8 ... 1 4 8]
0.8382682926829268
Iteration:  100
[6 9 8 ... 1 4 2] [6 9 8 ... 1 4 8]
0.8460731707317073
Iteration:  110
[6 9 8 ... 1 4 2] [6 9 8 ... 1 4 8]
0.854170731707317
Iteration:  120
[6 9 8 ... 1 4 2] [6 9 8 ... 1 4 8]
0.8596585365853658
Iteration:  130
[6 9 8 ... 1 4 2] [6 9 8 ... 1 4 8]
0.8641951219512195
Iteration:  140
[6 9 8 ... 1 4

In [37]:
test(W1, b1, W2, b2)

[2 0 4 7 6 9 3 8 7 0 9 9 1 0 9 0 5 0 1 0 3 1 1 5 9 4 6 0 7 0 9 2 4 9 2 0 2
 2 5 1 7 2 0 2 0 6 2 0 3 3 4 0 1 6 0 0 4 3 6 6 9 2 4 3 5 9 8 6 7 0 0 3 8 4
 2 4 7 7 7 1 3 2 9 7 1 3 4 1 4 4 8 8 2 6 4 3 6 1 8 4 6 6 7 0 3 3 1 2 7 2 6
 0 4 5 2 2 7 4 9 6 5 1 4 2 8 6 7 6 2 8 9 4 2 7 7 7 5 4 0 3 1 6 8 0 9 4 3 7
 6 5 1 4 5 5 7 7 3 4 4 8 8 0 0 6 7 2 3 5 1 9 7 7 1 1 6 8 4 1 7 6 8 5 5 9 3
 0 3 8 7 9 1 7 6 5 2 0 5 8 7 7 3 0 8 4 6 6 5 8 6 9 4 6 2 7 5 0 7 7 8 6 1 0
 0 9 2 0 3 7 1 1 6 3 2 8 4 3 2 5 6 8 6 7 8 3 0 2 6 4 5 5 5 8 2 8 3 5 9 1 2
 3 7 8 1 7 3 0 4 1 2 2 2 3 8 4 5 1 1 4 3 4 0 1 2 9 6 0 0 1 6 6 7 9 1 3 1 9
 5 5 1 9 3 3 2 1 4 9 0 2 6 9 0 8 7 7 4 0 8 3 3 8 7 4 1 1 4 4 3 7 7 7 1 2 1
 4 0 8 2 4 8 4 1 9 1 8 4 7 6 7 7 6 5 8 6 9 6 0 6 3 3 9 1 2 0 6 7 3 0 3 7 8
 0 3 0 1 4 8 8 2 6 8 0 6 2 5 2 6 9 0 7 0 3 4 9 7 8 2 8 7 9 6 0 0 6 1 0 1 2
 8 3 7 1 4 0 1 3 4 4 3 0 8 9 1 9 0 6 1 5 0 9 6 3 3 0 6 5 4 2 6 1 0 2 2 1 2
 4 5 3 9 5 5 1 0 6 6 2 8 5 1 3 5 1 4 4 9 7 3 7 5 9 4 0 3 0 9 5 2 0 8 1 9 3
 4 1 6 4 1 9 1 0 6 9 9 4 

In [38]:
only_predictions(W1, b1, W2, b2, X)

Probabilities of Class Predictions:
[[4.46208041e-14]
 [9.58606379e-03]
 [8.63032337e-01]
 [1.27084197e-01]
 [1.84370122e-13]
 [2.16891363e-10]
 [9.91030885e-09]
 [4.34583388e-08]
 [2.97311534e-04]
 [3.74289607e-08]]

Class Predictions:
[2]
